In [1]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from functools import reduce
from selenium import webdriver
# from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [1]:
def chk_nxt(driver):
    '''Checks if there is still a next button left to click.
    '''

    btns = driver.find_elements_by_class_name("btn-next-prev")
    btns = list(filter(lambda x: x.text.strip()=='Next', btns))

    if btns == []:
        return (False, btns)
    else:
        return (True, btns)

def scrape(driver):
    '''Scrapes the job ad details from the webdriver.
    
    Parameters:
    -----------
    driver: webdriver object
    '''

    # retrieve all the text in the title section
    title = driver.find_element_by_css_selector(".card-body.card-body-apply.pl15").text

    # retrieve text in job description portion
    jobd = driver.find_element_by_class_name("jd-text").text

    # retrieve text in li tags
    jobd_li = driver.find_element_by_class_name("jd-text").find_elements_by_tag_name('li') # returns a list of li elements
    li_list = [element.text for element in jobd_li]

    # retrieve data in job skills section
    skills = [element.text.strip() for element in driver.find_elements_by_class_name('round-card')]

    # you may or may not have li, skills
    tmp_dict = {'title':title, 'jobd':jobd, 'li_list':li_list, 'skills':skills}
    
    return tmp_dict

def clean_save(jobs_det, cntr):
    '''Do preliminary transformation of the scraped details then save as csv.
    
    Parameters:
    -----------
    jobs_det: list of dictionaries
        Details of each job post.
    '''
    # Get dictionary keys.
    k_list = jobs_det[0].keys()
    
    list_od = []
    for i, job in enumerate(jobs_det):
        try:
            for k in k_list:
                if k == 'title':
                    tmp_list = job[k].split('\n')
                    tmp_title = tmp_list[0].strip()
                    tmp_com = tmp_list[1].strip()
                    tmp_loc = tmp_list[2].strip()
                elif k == 'jobd':
                    tmp_rawd = job[k]
                elif k == 'li_list':
                    tmp_li = job[k]
                elif k == 'skills':
                    tmp_ski = ' '.join(job[k])
                else:
                    pass
            tmp_dict = {'title':tmp_title, 'company':tmp_com,
                        'location':tmp_loc,'description':tmp_rawd,
                        'bullets':tmp_li, 'k_words':tmp_ski}
            list_od.append(tmp_dict)
        except:
            continue

    df = pd.DataFrame(list_od)
    df = df[['title', 'company', 'location', 'description', 'bullets', 'k_words']]
    df.to_csv(r'Summaries/Monster/'+str(cntr)+'.csv', index=False)

In [3]:
# Open webdriver and access the page.
driver = webdriver.Chrome(r'C:\chromedriver')
driver.get('https://www.monster.com.ph/search/results?query=it,computers,hardware&limit=25&sort=2&industries=it%2fcomputers%20-%20software,telecom,it%2fcomputers%20-%20hardware&filter=true')

# User inputs:
cntr = 0
cntr_end = 50
sleep_t1 = 7 # Waiting time for elements to load

# Initialization
nxtbtn = True

# Check number of pages in monster job index and set that to max cntr val
while ((nxtbtn == True) and cntr <= cntr_end):
    
    sleep(sleep_t1)
    
    # Get latest url
    latest_ind = driver.current_url

    # Update exit clauses:
    cntr += 1
    nxtbtn, btns = chk_nxt(driver)
    print(btns)
    
    # Find all the job ad panels
    job_ads = driver.find_elements_by_class_name('card-body-apply')

    jobs_det = [] # Stores the job details scraped
    failed = [] # Stores the failed url links
    cnt_failed = 0
    cnt_scraped = 0

    # Scrape each job post
    for ad in job_ads:
        
        # Click job and switch
        ad.click()
        driver.switch_to.window(driver.window_handles[-1])

        sleep(sleep_t1)
        try:
            # Scrape then append the job details to jobs_det
            jobs_det.append(scrape(driver))
            cnt_scraped += 1
            
            # Close current tab, go back to main tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
        except:
            print('skipping..')
            cnt_failed += 1
            failed.append(driver.current_url)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

    # Clean and saves the job details for the current index page.
    clean_save(jobs_det, cntr)
    # Append latest link, click next button
    btns[0].click()

[<selenium.webdriver.remote.webelement.WebElement (session="345f76a86400661f0944e9588b0409d7", element="0.5713203449691695-1")>]
skipping..
skipping..
skipping..
skipping..
[<selenium.webdriver.remote.webelement.WebElement (session="345f76a86400661f0944e9588b0409d7", element="0.5713203449691695-1")>]
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
[<selenium.webdriver.remote.webelement.WebElement (session="345f76a86400661f0944e9588b0409d7", element="0.5713203449691695-1")>]
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
skipping..
[<selenium.webdriver.remote.webelement.WebElement (session="345f76a86400661f0944e9588b0409d7", element="0.5713203449691695-1")>]
[<selenium.webdriver.remote.webelement.WebElement (session="345f76a86400661f0944e9588b0409d7", element="0.5713203449691695-1")>]
skipping..
skipping..
skipping..
[<selenium.webdriver.remote.webelement.WebElement (session="345f76a86

KeyboardInterrupt: 

In [8]:
cntr

14

In [10]:
latest_ind[-1]

'https://www.monster.com.ph/search/results?start=600&sort=2&limit=25&query=it,computers,hardware,it,computers,software'